<a href="https://colab.research.google.com/github/DivyanshSareen/movi_rec/blob/main/movie_recc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [106]:
import requests
import pandas as pd

In [ ]:
api_key = ""    # tmdb api key here

In [ ]:
# demo request
response = requests.get("https://api.themoviedb.org/3/movie/top_rated?api_key="+api_key)
response
type(response.json())

dict

In [117]:
# utility function
def get_cast(credits):
  cast = credits['cast']
  act_li = []
  i = 0
  while i < 3:
    if cast[i]['known_for_department'] == "Acting":
      act_li.append(cast[i]['name'])
    i = i + 1
  return act_li

def get_direc(credits):
  cast = credits['crew']
  direc_li = []
  i = 0
  for x in cast:
    if 'job' in x:
      direc_li.append(x['name'])
      i = i + 1
    if i < 2: break
  return direc_li

# returns a list of attributes corresponding to given movie id
def movie_row(id):
  # request url for single movie
  url = "https://api.themoviedb.org/3/movie/"+str(id)+"?api_key="+api_key+"&language=en-US&append_to_response=credits"
  response = requests.get(url)
  data = response.json()
  # print(data.keys())
  genres = [x['name'] for x in data['genres']]
  overview = data['overview']
  title = data['title']
  cast = get_cast(data['credits'])
  director = get_direc(data['credits'])
  # print(director)
  return [id, title, genres, overview, cast, director]

In [119]:
# request URL for API
url = "https://api.themoviedb.org/3/movie/top_rated?api_key="+api_key+"&language=en-US&page="

# list for movie dataframe
df_li = []

# list of pages with issue
error_pages = []

# iterating over pages 1 to 19 
for i in range(1,25):
  response = requests.get(url+str(i))    # getting response for ith page
  data = response.json()   
  # iterating over all movies in a page 
  for j in data['results']:
    # print(j['title'])
    try:
      row_li = movie_row(j['id'])
      print(row_li)
      df_li.append(row_li)
    except:
      error_pages.append(j)
    
print("Process complete! error in these pages:")
for i in error_pages:
  print(i)

df = pd.DataFrame(df_li, columns = ['id', 'title', 'genre', 'overview', 'cast', 'director'])

[19404, 'Dilwale Dulhania Le Jayenge', ['Comedy', 'Drama', 'Romance'], 'Raj is a rich, carefree, happy-go-lucky second generation NRI. Simran is the daughter of Chaudhary Baldev Singh, who in spite of being an NRI is very strict about adherence to Indian values. Simran has left for India to be married to her childhood fiancé. Raj leaves for India with a mission at his hands, to claim his lady love under the noses of her whole family. Thus begins a saga.', ['Shah Rukh Khan', 'Kajol', 'Amrish Puri'], ['Farah Khan']]
[278, 'The Shawshank Redemption', ['Drama', 'Crime'], 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', ['Tim Robbins', 'Morgan Freeman', 

In [120]:
df.head()

,id,title,genre,overview,cast,director
0,19404,Dilwale Dulhania Le Jayenge,"[Comedy, Drama, Romance]","Raj is a rich, carefree, happy-go-lucky second...","[Shah Rukh Khan, Kajol, Amrish Puri]",[Farah Khan]
1,278,The Shawshank Redemption,"[Drama, Crime]",Framed in the 1940s for the double murder of h...,"[Tim Robbins, Morgan Freeman, Bob Gunton]",[Thomas Newman]
2,238,The Godfather,"[Drama, Crime]","Spanning the years 1945 to 1955, a chronicle o...","[Marlon Brando, Al Pacino, James Caan]",[Francis Ford Coppola]
3,724089,Gabriel's Inferno Part II,[Romance],Professor Gabriel Emerson finally learns the t...,"[Melanie Zanetti, Giulio Berruti, James Andrew...",[Denis Maloney]
4,761053,Gabriel's Inferno Part III,"[Romance, Comedy]",The final part of the film adaption of the ero...,"[Melanie Zanetti, Giulio Berruti, Rhett Wellin...",[Tosca Musk]


In [121]:
df.to_csv("tmdb_movie.csv")

In [122]:
df.shape

(469, 6)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval

In [ ]:
md = pd. read_csv('/content/drive/MyDrive/data_movies/movies_metadata.csv')
md.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [ ]:
m = vote_counts.quantile(0.95)
m

434.0

In [ ]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'vote_count', 'vote_average', 'popularity']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 4)

v is the number of votes for the movie

m is the minimum votes required to be listed in the chart

R is the average rating of the movie

C is the mean vote across the whole report



In [ ]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [ ]:
qualified = qualified.sort_values('wr', ascending=False).head(25)

In [ ]:
print(qualified['title'])

15480                                            Inception
12481                                      The Dark Knight
22879                                         Interstellar
2843                                            Fight Club
4863     The Lord of the Rings: The Fellowship of the Ring
292                                           Pulp Fiction
314                               The Shawshank Redemption
7000         The Lord of the Rings: The Return of the King
351                                           Forrest Gump
5814                 The Lord of the Rings: The Two Towers
256                                              Star Wars
1225                                    Back to the Future
834                                          The Godfather
1154                               The Empire Strikes Back
46                                                   Se7en
24860                                   The Imitation Game
359                                          The Lion Ki

In [ ]:
links_small = pd.read_csv('/content/drive/MyDrive/data_movies/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
md = md.drop([19730, 29503, 35587])

In [ ]:
md['id'] = md['id'].astype('int')

In [ ]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

In [ ]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
credits = pd.read_csv('/content/drive/MyDrive/data_movies/credits.csv')

In [ ]:
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [ ]:
smd = smd.merge(credits, on='id')

In [ ]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
smd['director'] = smd['crew'].apply(get_director)

In [ ]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline'] + smd['director']+ smd['director']+ smd['director']+ smd['director']
smd['description'] = smd['description'].fillna('')

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [ ]:
tfidf_matrix.shape

(9133, 294102)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

array([1.        , 0.00568216, 0.00250467, ..., 0.        , 0.        ,
       0.00412714])

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles[movie_indices]

In [ ]:
s = get_recommendations('The Family').head(10)
for i in s:
  print(i)

The Fifth Element
La Femme Nikita
Lucy
The Messenger: The Story of Joan of Arc
Leon: The Professional
Arthur and the Invisibles
The Big Blue
Did You Hear About the Morgans?
Charlie's Angels: Full Throttle
The Whole Nine Yards


In [ ]:
s = get_recommendations('Cool as Ice').head(10)
for i in s:
  print(i)

Inspector Gadget
Did You Hear About the Morgans?
The Favor
Charlie's Angels: Full Throttle
The Family
The Whole Nine Yards
Eraser
House of Sand and Fog
Cry-Baby
My Blue Heaven


In [ ]:
s = get_recommendations("Charlie's Angels: Full Throttle").head(10)
for i in s:
  print(i)

Did You Hear About the Morgans?
The Family
Eraser
Wings of Desire
My Blue Heaven
The Whole Nine Yards
Cool as Ice
To Live and Die in L.A.
See Spot Run
Angels in the Outfield


In [ ]:
smd.to_csv('s_data.csv')